<a href="https://colab.research.google.com/github/syaifulalam61/strata/blob/main/convexOpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
import scipy.sparse as sp
import statsmodels.api as sm
from numpy.fft import fft, fftfreq
import plotly.graph_objects as go

In [ ]:
'''
scripted by Syaiful Alam
'''
###
'''
The data format consists of two columns, with the first column representing
the sampling interval and the second column containing the data series, such as
lithology, δ18O, or other stratigraphic proxies.
'''
# Load data
data = 'your/path'

# Convex Optimization
lambda_reg = 0.1
window_size = 0.01
n_values = x_values.size
n_long = x_values.size
a_matrix = np.ones((1, n_long))
b_matrix = np.vstack((a_matrix, -2 * a_matrix, a_matrix))
c_matrix = range(3)
d_matrix = sp.spdiags(b_matrix, c_matrix, n_long - 2, n_long)

# Optimization
s = cp.Variable(n_values)
s_long = cp.Variable(n_long)
objective = cp.Minimize(window_size * cp.sum_squares(x_values - s_long) +
                        lambda_reg * cp.norm(d_matrix @ s_long, 1))
problem = cp.Problem(objective)
problem.solve()

if problem.status != cp.OPTIMAL:
    raise Exception("Solver did not converge!")

# Yule-Walker Estimation
yule_walker_estimate = x_values - np.array(s_long.value)
order = 5
yule_walker_rho, yule_walker_sigma = sm.regression.yule_walker(yule_walker_estimate, order=order, method='mle')

# Create Yule-Walker Equation
yule_walker_estimate = pd.DataFrame(yule_walker_estimate)
yule_walker_equation = (yule_walker_rho[0] * yule_walker_estimate.shift(-1) +
                        yule_walker_rho[1] * yule_walker_estimate.shift(-2) +
                        yule_walker_rho[2] * yule_walker_estimate.shift(-3) +
                        yule_walker_rho[3] * yule_walker_estimate.shift(-4) +
                        yule_walker_rho[4] * yule_walker_estimate.shift(-5))

# Combine Results
depths = data[:, 0]
combined_results = np.column_stack((depths, yule_walker_estimate, yule_walker_equation))

yule_walker_nan = yule_walker_equation.to_numpy()
predictions = yule_walker_nan[~np.isnan(yule_walker_nan).any(axis=1)]

def count_non_nan(values):
    return np.sum(~np.isnan(values))

yule_walker_origin = yule_walker_estimate[0:len(yule_walker_estimate) - (len(yule_walker_nan) - count_non_nan(yule_walker_nan))]
depths_origin = depths[0:len(depths) - (len(yule_walker_nan) - count_non_nan(yule_walker_nan))]
origin_stack = np.column_stack((depths_origin, yule_walker_origin, predictions))

header = ['Depth', 'Yule-Walker Origin', 'Predictions']
final_results = pd.DataFrame(origin_stack, columns=header)
errors = final_results['Predictions'] - final_results['Yule-Walker Origin']
final_results = np.column_stack((depths_origin, yule_walker_origin, predictions, errors))
header = ['Depth', 'Yule-Walker Origin', 'Predictions', 'Prediction Error']
final_results = pd.DataFrame(final_results, columns=header)
cumulative_errors = final_results['Prediction Error'].cumsum()
final_results = np.column_stack((depths_origin, yule_walker_origin, predictions, errors, cumulative_errors))
header = ['Depth', 'Yule-Walker Origin', 'Predictions', 'Prediction Error', 'Cumulative Error']
final_results = pd.DataFrame(final_results, columns=header)
final_results_array = np.array(final_results)

# FFT Calculation
mean_fft = np.mean(final_results_array[:, 4])
std_fft = np.std(final_results_array[:, 4])
len_fft = len(final_results_array[:, 4])
frequencies_fft = fftfreq(len_fft)
mask = frequencies_fft > 0
period_fft = 1 / frequencies_fft[mask]
period_fft = period_fft / len(final_results_array[:, 4])
fft_values = fft(final_results_array[:, 4])
power_fft = 2 * (np.abs(fft_values / len_fft)**2)[mask]
header_fft = ['Frequency', 'Power Spectrum']
spectrum_fft = pd.DataFrame(np.column_stack((frequencies_fft[mask], power_fft)), columns=header_fft)
half_spectrum_fft = int(round(len(power_fft) / 2))

# Plot FFT using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=frequencies_fft[mask][:half_spectrum_fft],
    y=power_fft[:half_spectrum_fft],
    mode='lines',
    line=dict(color='dimgrey'),
    name='Power Spectrum'
))

fig.update_layout(
    title='FFT Analysis (Short Record)',
    xaxis_title='Frequency',
    yaxis_title='Power Spectrum',
    width=600,
    height=400
)

fig.show()